In [98]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup
import pandas as pd
import os.path
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

download_dir = input('Введите директорию для загрузки файлов. Туда сохранятся файлы, необходимые для работы программы.')
chrdr = input('Введите директорию, в которой хранится вебдрайвер')
print('Устанавливаю настройки браузера...')
options = wd.ChromeOptions()

profile = {"plugins.always_open_pdf_externally": True,
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)

driver = wd.Chrome(chrdr, options=options)
print('Все готово к парсингу! Запустите следующую ячеку')

print('Начинаю парсинг Economic Freedom...')
pdf_url = 'https://www.heritage.org/index/download?version=821#'
driver.get(pdf_url)
driver.find_element_by_css_selector('#show-older-editions').click()
for i in [2,5,8]:
    driver.find_element_by_css_selector(f'#older-editions > a:nth-child({i})').click()

print('Проверяю, скачались ли файлы...')

while not os.path.exists(download_dir+f'\index2017_data.xls'):
    time.sleep(1)
if os.path.isfile(download_dir+f'\index2017_data.xls'):
    years = [2017,2018,2019]
    all_datasets = {year:pd.read_excel(download_dir+f'\index'+str(year)+'_data.xls') for year in years}

print('Собираю датафрейм...')
    
countries = []
index = []
years2 = []
for year, data in zip(years, all_datasets.values()):
    for cun, ind in zip(data['Country Name'], data[f'{year} Score']):
        countries.append(cun)
        index.append(ind)
        years2.append(year)
        
EFI = pd.DataFrame({'Year':years2, 'Index of Economic Freedom':index}, index=countries)
print('EFI готов!')

print('Начинаю парсинг government effectiveness...')

link = 'https://govdata360.worldbank.org/indicators/h580f9aa5?country=BRA&indicator=388&viz=line_chart&years=1996,2019#table-link'
driver.get(link)
driver.switch_to.alert.accept()

print('Ожидаю полной загрузки страницы...')

test = '<td>Brazil</td>'
while test not in str(BeautifulSoup(driver.page_source)):
    time.sleep(1)
print('Страница загружена!')
print('Начинаю цикл парсинга 1 из 3...')
countries = []
index_2019 = []
year = []
for i in range(1,4):
    num = 0
    for_it = 1
    while num != 41:
        cun = BeautifulSoup(driver.page_source).find_all('td')[::5]
        index = BeautifulSoup(driver.page_source).find_all('td')[i::5]
        for cy, ind in zip(cun, index):
            countries.append(cy.get_text())
            index_2019.append(ind.get_text())
            if i == 1:
                year.append(2017)
            elif i == 2:
                year.append(2018)
            elif i == 3:
                year.append(2019)
        num += 1
        if num <= 4:
            driver.find_element_by_css_selector(f'#DataTables_Table_0_paginate > span > a:nth-child({num+1})').click()
        elif num > 4 and num <= 38:
            driver.find_element_by_css_selector('#DataTables_Table_0_paginate > span > a:nth-child(5)').click()
        elif num > 38 and num != 41:
            driver.find_element_by_css_selector(f'#DataTables_Table_0_paginate > span > a:nth-child({5+for_it})').click()
            for_it += 1
    print(f'Цикл {i} из 3 закончен')
    driver.find_element_by_css_selector('#DataTables_Table_0_paginate > span > a:nth-child(1)').click()

government_effectivness = pd.DataFrame({'Year':year, 'Government Effectivness Index':index_2019}, index=countries)

print("Я спарсил Government Effectivness!")

print('Начинаю парсить Economic complexity index...')

driver = wd.Chrome(chrdr, options=options)
url = 'https://oec.world/en/rankings/eci/hs6/hs12'
driver.get(url)
test = '<div class="name">Japan</div>'

print('Жду полной загрузки страницы...')
while test not in str(BeautifulSoup(driver.page_source)):
    time.sleep(1)
    
print('Страница готова!')
    
countries = BeautifulSoup(driver.page_source).find_all('div', {'class': 'name'})
country_names = [cnt.get_text() for cnt in countries]
item_cell = BeautifulSoup(driver.page_source).find_all('div', {'class':'value'})
item_2018 = [i.get_text() for i in item_cell]
OEC = pd.DataFrame(zip(country_names, item_2018), columns = ['Country', 'Value_2018'])
OEC['year'] = [2018]*len(OEC['Country'])

print('Со сложностью экономики все!')
print('На очереди World Bank...')

url='https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?most_recent_year_desc=true&view=chart'
driver.get(url)

print('Жду загрузку страницы...')

delay = 3 # seconds
test_css = '#main > div > div > article > section > div > div > div > section > div > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a'
try:
    myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, test_css)))
    print ("Страница готова!")
except TimeoutException:
    print ("Загрузка слишком долгая :(")
    
countries = BeautifulSoup(driver.page_source).find_all('a', {'class': 'country-name'})
country_names = [cnt.get_text() for cnt in countries]

count = []
item = BeautifulSoup(driver.page_source).find_all('div', {'class':'item'})
item_1 = [i.find_all('div') for i in item]
for country in item_1:
    info_1 = [i.get_text() for i in country]
    count.append(info_1)
    
WB = pd.DataFrame(count)[[0,1,2]]
print('Парсинг завершен!')
print('Закрываю браузер')

driver.quit()

print('Начинаю сводить единый датасет...')

av_c = set(OEC['Country']) & set(EFI.index.tolist()) & set(government_effectivness.index.tolist()) & set(WB[0])
EFI1 = EFI.copy()
government_effectivness1 = government_effectivness.copy()
OEC1 = OEC.copy()
WB1 = WB.copy()
EFI1['Country'] = EFI1.index.tolist()
government_effectivness1['Country'] = government_effectivness1.index.tolist()
WB1 = WB1.rename(columns={0:'Country', 1:'Year', 2:'GDP'})
for frame in [EFI1, government_effectivness1, OEC1, WB1]:
    check = frame.Country.isin(list(av_c))
    frame = frame[check]
to_cnct = [EFI1, OEC1, WB1]
cntr = pd.merge(EFI1, OEC1, on='Country')
Economic = pd.merge(cntr, WB1, on='Country')
Economic
full_set = {'1':government_effectivness1, '2':Economic}

print('Датасет собран! Выгружаю данные...')

Economic.to_excel(download_dir+'\economic.xlsx')
government_effectivness1.to_excel(download_dir+'\political.xlsx')

print('Готово! Для работы с данными загрузите следующую ячейку')

Введите директорию для загрузки файлов. Туда сохранятся файлы, необходимые для работы программы.C:\Users\bendr\Documents\chrdr
Введите директорию, в которой хранится вебдрайверC:\Users\bendr\Documents\chrdr\chromedriver.exe
Устанавливаю настройки браузера...
Все готово к парсингу! Запустите следующую ячеку
Начинаю парсинг Economic Freedom...
Проверяю, скачались ли файлы...
Собираю датафрейм...
EFI готов!
Начинаю парсинг government effectiveness...
Ожидаю полной загрузки страницы...
Страница загружена!
Начинаю цикл парсинга 1 из 3...
Цикл 1 из 3 закончен
Цикл 2 из 3 закончен
Цикл 3 из 3 закончен
Я спарсил Government Effectivness!
Начинаю парсить Economic complexity index...
Жду полной загрузки страницы...
Страница готова!
Со сложностью экономики все!
На очереди World Bank...
Жду загрузку страницы...
Страница готова!
Парсинг завершен!
Закрываю браузер
Начинаю сводить единый датасет...
Датасет собран! Выгружаю данные...
